In [13]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
from pathlib import Path
from QligFEP.pdb_utils import (
    nest_pdb,
    unnest_pdb,
    read_pdb_to_dataframe,
    write_dataframe_to_pdb,
)
from QligFEP.CLI.pdb_to_amber import asp_search
import pandas as pd

# Define functions

In [15]:
# Need to remove Hs from GLY
# Need to cap the last residue

rename_mapping = {
    "ARG": {
        "1HH1": "HH11",
        "2HH1": "HH12",
        "1HH2": "HH21",
        "2HH2": "HH22",
        "HA2": "HA3",
        "HA1": "HA2",
        "HB2": "HB3",
        "HB1": "HB2",
        "HG2": "HG3",
        "HG1": "HG2",
        "HD2": "HD3",
        "HD1": "HD2",
    },
    "ILE": {
        "1HG2": "HG21",
        "2HG2": "HG22",
        "3HG2": "HG23",
        # HG1 with has the number +1 in our naming scheme
        "1HG1": "HG12",
        "2HG1": "HG13",
        # this is fine...
        "CD": "CD1",
        "HD1": "HD11",
        "HD2": "HD12",
        "HD3": "HD13",
    },
    "THR": {
        "1HG2": "HG21",
        "2HG2": "HG22",
        "3HG2": "HG23",
    },
    "LEU": {
        "1HD1": "HD11",
        "2HD1": "HD12",
        "3HD1": "HD13",
        "1HD2": "HD21",
        "2HD2": "HD22",
        "3HD2": "HD23",
        "HA2": "HA3",
        "HA1": "HA2",
        "HB2": "HB3",
        "HB1": "HB2",
    },
    "GLN": {
        "1HE2": "HE21",
        "2HE2": "HE22",
        "HA2": "HA3",
        "HA1": "HA2",
        "HB2": "HB3",
        "HB1": "HB2",
        "HG2": "HG3",
        "HG1": "HG2",
    },
    "GLY": {
        "HA2": "HA3",
        "HA1": "HA2",
    },
    "VAL": {
        "1HG1": "HG11",
        "2HG1": "HG12",
        "3HG1": "HG13",
        "1HG2": "HG21",
        "2HG2": "HG22",
        "3HG2": "HG23",
    },
    "SER": {
        "HB2": "HB3",
        "HB1": "HB2",
    },
    "PHE": {
        "HB2": "HB3",
        "HB1": "HB2",
    },
    "GLU": {
        "HB2": "HB3",
        "HB1": "HB2",
        "HG2": "HG3",
        "HG1": "HG2",
    },
    "ASP": {
        "HB2": "HB3",
        "HB1": "HB2",
    },
    "ASH": {
        "HB2": "HB3",
        "HB1": "HB2",
    },
    "ASN": {
        "HB2": "HB3",
        "HB1": "HB2",
        "1HD2": "HD21",
        "2HD2": "HD22",
    },
    "LYS": {
        "HB2": "HB3",
        "HB1": "HB2",
        "HG2": "HG3",
        "HG1": "HG2",
        "HD2": "HD3",
        "HD1": "HD2",
        "HE2": "HE3",
        "HE1": "HE2",
    },
    "PRO": {
        "HB2": "HB3",
        "HB1": "HB2",
        "HD2": "HD3",
        "HD1": "HD2",
        "HG2": "HG3",
        "HG1": "HG2",
    },
    "MET": {
        "HB2": "HB3",
        "HB1": "HB2",
        "HG2": "HG3",
        "HG1": "HG2",
    },
    "TYR": {
        "HB2": "HB3",
        "HB1": "HB2",
    },
    "HIE": {
        "HB2": "HB3",
        "HB1": "HB2",
    },
    "HIP": {
        "HB2": "HB3",
        "HB1": "HB2",
    },
    "HID": {
        "HB2": "HB3",
        "HB1": "HB2",
    },
    "TRP": {
        "HB2": "HB3",
        "HB1": "HB2",
    },
    "CYS": {
        "HB2": "HB3",
        "HB1": "HB2",
    },
    "CYX": {
        "HB2": "HB3",
        "HB1": "HB2",
    },
    "NMA": {
        "CH3": "CA",
        "1HH3": "HA1",
        "2HH3": "HA2",
        "3HH3": "HA3",
    },
    "ACE": {
        "1HH3": "HH31",
        "2HH3": "HH32",
        "3HH3": "HH33",
    },
}


def correct_amino_acid_atom_names(npdb_i, resname, rename_mapping):
    """corrects the amino acid atom names according to the mapping provided

    Args:
        npdb_i: nested pdb data structure for a single residue
        resname: the residue name
        rename_mapping: a dictionary mapping old names to new names
    """
    if resname in rename_mapping:
        for old_name, new_name in rename_mapping[resname].items():
            npdb_i = [extract_and_replace(x, old_name, new_name) for x in npdb_i]
            # certify that we have the alignment as expected for pdb files
    return npdb_i


def extract_and_replace(line, old_name, new_name):
    """extracts the atom name and replaces it with the new name"""
    atom_name = line[12:16].strip()
    new_atom_name = atom_name.replace(old_name, new_name).strip()
    if len(new_atom_name) == 4:
        return line[:12] + new_atom_name + line[16:]
    else:
        # return left aligned atom name always with len() == 3 but with a " " in the beginning
        return line[:12] + f" {new_atom_name:<3}" + line[16:]


def fix_pdb(pdb_path: Path, rename_mapping):
    renamed_pdb_path = pdb_path.with_name(pdb_path.stem + "_renamed.pdb")
    with open(pdb_path) as f:
        pdb_lines = f.readlines()

    npdb = nest_pdb(pdb_lines)
    npdb = asp_search(npdb)

    for i, res in enumerate(npdb):
        resname = res[-1][17:20]
        if resname == "HIS":  # rename to HIP according to our FF library
            npdb[i] = [x.replace("HIS", "HIP") for x in npdb[i]]
            resname = "HIP"
        if resname == "NME":  # we use NMA in our FF library
            npdb[i] = [x.replace("NME", "NMA") for x in npdb[i]]
            resname = "NMA"
        npdb[i] = correct_amino_acid_atom_names(npdb[i], resname, rename_mapping)
    pdb_lines = unnest_pdb(npdb)

    with open(renamed_pdb_path, "w") as f:
        for line in pdb_lines:
            f.write(line)
    return pdb_lines


def cap_and_reindex_pdb(inp_pdb: Path):
    """Function that removes additionaly hydrogens from N terminal not covered
    in our library files, caps the last residue and reindexes the atoms

    Args:
        inp_pdb: path for the pdb file
    """

    pdb_df = read_pdb_to_dataframe(inp_pdb)

    # remove extra Hs from the first Gly residue
    if pdb_df["residue_name"].values[0] in ["GLY", "LEU", "GLU", "ASH", "ILE", "ASN"]:
        first_residue = pdb_df["residue_seq_number"].values[0]
        # remove atoms with atom_name H2 and H3, and rename H1 to H
        subset_first = pdb_df[
            (pdb_df["residue_seq_number"] == first_residue)
            & (~pdb_df["atom_name"].isin(["H2", "H3"]))
        ].copy()
        subset_first["atom_name"] = subset_first["atom_name"].str.replace("H1", "H")
        rm_idxs = pdb_df.query("residue_seq_number == @first_residue").index
        pdb_df = pd.concat(
            [subset_first, pdb_df.drop(index=rm_idxs)], ignore_index=True
        )
    # cap the last residue
    last_residue = pdb_df["residue_name"].values[-1]
    last_residue_number = pdb_df["residue_seq_number"].values[-1]  # noqa: F841
    if last_residue in ["ILE", "NME"]:
        rm_idxs = pdb_df.query("residue_seq_number == @last_residue_number").index
        pdb_df.drop(index=rm_idxs, inplace=True)
    _len = len(pdb_df)
    pdb_df["atom_serial_number"] = range(1, _len + 1)
    write_dataframe_to_pdb(pdb_df, inp_pdb)

# Rename the protein files

In [16]:
pdb_paths = sorted(Path().glob("*/protein/protein.pdb"))

In [17]:
for pdb_path in pdb_paths:
    fix_pdb(pdb_path, rename_mapping)
    cap_and_reindex_pdb(pdb_path.with_stem(pdb_path.stem + "_renamed"))

# Rename the water & cofactor files

In [18]:
atom_renaming_dict = {
    # water mols
    "HW1": "H1",
    "HW2": "H2",
    "OW": "O",
    # salts
    "ClJ": "CHL",
    "NA": "SOD",
    "MG": "MAG",
    "ZN": "ZIN",
}
residue_renaming_dict = {
    "ClJ": "CHL",
    "NA": "SOD",
    "MG": "MAG",
    "ZN": "ZIN",
}


def rename_waters_atoms(pdb_path: Path, atom_renaming_dict: dict):
    pdb_df = read_pdb_to_dataframe(pdb_path).assign(
        atom_name=lambda x: x["atom_name"].replace(atom_renaming_dict),
        residue_name=lambda x: x["residue_name"]
        .str.strip(" ")
        .replace(residue_renaming_dict),
    )
    write_dataframe_to_pdb(pdb_df, pdb_path.with_stem(pdb_path.stem + "_renamed"))

In [19]:
cofactor_paths = sorted(Path().glob("*/protein/cofactors_crystalwater.pdb"))

for pdb_path in cofactor_paths:
    if pdb_path.stat().st_size != 0:
        rename_waters_atoms(pdb_path, atom_renaming_dict)

# Merge renamed protein & cofactor files

In [20]:
prot_root_paths = sorted(Path().glob("*/protein/"))

for _path in prot_root_paths:
    processed_pdbs = []
    protfile = _path / "protein_renamed.pdb"
    cofactor = _path / "cofactors_crystalwater_renamed.pdb"

    prot_df = read_pdb_to_dataframe(protfile)

    # reindex both atom_serial_number and residue_seq_number
    prot_df["atom_serial_number"] = range(1, len(prot_df) + 1)
    residue_seq_mapping = {
        old: new
        for old, new in zip(
            prot_df["residue_seq_number"].unique(),
            range(1, len(prot_df["residue_seq_number"].unique()) + 1),
        )
    }
    prot_df["residue_seq_number"] = prot_df["residue_seq_number"].replace(
        residue_seq_mapping
    )
    last_prot_res = prot_df["residue_seq_number"].max()
    last_prot_atom = prot_df["atom_serial_number"].max()
    processed_pdbs.append(prot_df)

    if cofactor.exists():
        print("Including cofactors for ", _path)
        cof_df = read_pdb_to_dataframe(cofactor)
        cof_df["atom_serial_number"] = range(
            last_prot_res + 1, last_prot_res + len(cof_df) + 1
        )
        residue_seq_mapping = {
            old: new
            for old, new in zip(
                cof_df["residue_seq_number"].unique(),
                range(
                    last_prot_res + 1,
                    last_prot_res + len(cof_df["residue_seq_number"].unique() + 1),
                ),
            )
        }
        cof_df["residue_seq_number"] = cof_df["residue_seq_number"].replace(
            residue_seq_mapping
        )
        processed_pdbs.append(cof_df)
    final_df = pd.concat(processed_pdbs, ignore_index=True)
    write_dataframe_to_pdb(final_df, protfile.parent / "protfile_final.pdb")

Including cofactors for  bace/protein
Including cofactors for  bace_hunt/protein
Including cofactors for  bace_p2/protein
Including cofactors for  cdk8/protein
Including cofactors for  eg5/protein
Including cofactors for  galectin/protein
Including cofactors for  hif2a/protein
Including cofactors for  mcl1/protein
Including cofactors for  p38/protein
Including cofactors for  pde10/protein
Including cofactors for  pde2/protein
Including cofactors for  pfkfb3/protein
Including cofactors for  ptp1b/protein
Including cofactors for  shp2/protein
Including cofactors for  syk/protein
Including cofactors for  thrombin/protein
Including cofactors for  tnks2/protein


# Preparing data

running qprep through the notebook. First we get the Center of Geometry (COG) of all the ligands, and then use it prepare the water spheres of the respective systems. The COG is the center of the water sphere

In [21]:
import shutil
from QligFEP.CLI.qprep_cli import main, QprepError
from QligFEP.CLI.cog_cli import MolecularCOG
import argparse
import os

prot_root_paths = sorted([p.absolute() for p in Path().glob("*/protein/")])
cwd = Path.cwd()

for _path in prot_root_paths:
    print("Processing: ", _path)
    processed_pdbs = []
    protfile = _path / "protfile_final.pdb"
    qprep_dir = _path / "qprep"
    if not qprep_dir.exists():
        qprep_dir.mkdir()
    shutil.copy(protfile, qprep_dir / "protein.pdb")

    # change the working directory to the qprep directory
    os.chdir(qprep_dir)

    # calculate the center of geometry for the ligands
    ligpath = _path.parent / "ligands/ligands.sdf"
    cog = MolecularCOG(ligpath)
    coords_str = cog()
    coordinates = [n for n in coords_str.strip("[]").split()]

    args = argparse.Namespace()
    args.log_level = "info"
    args.input_pdb_file = "protein.pdb"
    args.FF = "AMBER14sb"
    args.cog = coordinates
    args.sphereradius = 25
    args.cysbond = "auto"
    args.solvent_pack = 3.0

    try:
        main(args)
    except QprepError:
        continue

os.chdir(cwd)

2024-07-30 08:17:39 | WARNING  | QligFEP.CLI.cog_cli:_cog_sdf:66 - Calculating for all ligands in the file.
2024-07-30 08:17:39 | WARNING  | QligFEP.CLI.utils:cysbonds_for_qprep:35 - Disulfide bonds detected on CYS residues! Renaming to CYX


Processing:  /zfsdata/data/david/qligfepv2-BenchmarkExperiments/startFiles/bace/protein


2024-07-30 08:17:39 | WARNING  | QligFEP.CLI.qprep_cli:main:229 - qprep.inp already exists!! Overwriting...
STOP qprep ended normally
2024-07-30 08:17:40 | INFO     | QligFEP.CLI.qprep_cli:main:237 - qprep run finished. Check the output `qprep.out` for more information.
2024-07-30 08:17:40 | INFO     | QligFEP.CLI.qprep_cli:main:260 - water.pdb file created.
2024-07-30 08:17:40 | WARNING  | QligFEP.CLI.qprep_cli:main:270 - Found 1 water molecules outside the sphere radius.
2024-07-30 08:17:40 | WARNING  | QligFEP.CLI.qprep_cli:main:271 - Removing these water molecules from the water.pdb file.
2024-07-30 08:17:41 | WARNING  | QligFEP.CLI.cog_cli:_cog_sdf:66 - Calculating for all ligands in the file.
2024-07-30 08:17:41 | WARNING  | QligFEP.CLI.utils:cysbonds_for_qprep:35 - Disulfide bonds detected on CYS residues! Renaming to CYX


Processing:  /zfsdata/data/david/qligfepv2-BenchmarkExperiments/startFiles/bace_hunt/protein


2024-07-30 08:17:42 | WARNING  | QligFEP.CLI.qprep_cli:main:229 - qprep.inp already exists!! Overwriting...
STOP qprep ended normally
2024-07-30 08:17:43 | INFO     | QligFEP.CLI.qprep_cli:main:237 - qprep run finished. Check the output `qprep.out` for more information.
2024-07-30 08:17:43 | INFO     | QligFEP.CLI.qprep_cli:main:260 - water.pdb file created.
2024-07-30 08:17:43 | WARNING  | QligFEP.CLI.qprep_cli:main:270 - Found 15 water molecules outside the sphere radius.
2024-07-30 08:17:43 | WARNING  | QligFEP.CLI.qprep_cli:main:271 - Removing these water molecules from the water.pdb file.
2024-07-30 08:17:43 | WARNING  | QligFEP.CLI.cog_cli:_cog_sdf:66 - Calculating for all ligands in the file.
2024-07-30 08:17:43 | WARNING  | QligFEP.CLI.utils:cysbonds_for_qprep:35 - Disulfide bonds detected on CYS residues! Renaming to CYX


...

2024-07-30 08:18:10 | WARNING  | QligFEP.CLI.qprep_cli:main:229 - qprep.inp already exists!! Overwriting...
STOP qprep ended normally
2024-07-30 08:18:11 | INFO     | QligFEP.CLI.qprep_cli:main:237 - qprep run finished. Check the output `qprep.out` for more information.
2024-07-30 08:18:11 | INFO     | QligFEP.CLI.qprep_cli:main:260 - water.pdb file created.
2024-07-30 08:18:11 | INFO     | QligFEP.CLI.qprep_cli:main:280 - All water molecules are inside the sphere radius.
2024-07-30 08:18:11 | WARNING  | QligFEP.CLI.cog_cli:_cog_sdf:66 - Calculating for all ligands in the file.


Processing:  /zfsdata/data/david/qligfepv2-BenchmarkExperiments/startFiles/tnks2/protein


2024-07-30 08:18:12 | WARNING  | QligFEP.CLI.qprep_cli:main:229 - qprep.inp already exists!! Overwriting...
STOP qprep ended normally
2024-07-30 08:18:12 | INFO     | QligFEP.CLI.qprep_cli:main:237 - qprep run finished. Check the output `qprep.out` for more information.
2024-07-30 08:18:12 | INFO     | QligFEP.CLI.qprep_cli:main:260 - water.pdb file created.
2024-07-30 08:18:12 | WARNING  | QligFEP.CLI.qprep_cli:main:270 - Found 12 water molecules outside the sphere radius.
2024-07-30 08:18:12 | WARNING  | QligFEP.CLI.qprep_cli:main:271 - Removing these water molecules from the water.pdb file.
2024-07-30 08:18:13 | WARNING  | QligFEP.CLI.cog_cli:_cog_sdf:66 - Calculating for all ligands in the file.
2024-07-30 08:18:13 | WARNING  | QligFEP.CLI.qprep_cli:main:229 - qprep.inp already exists!! Overwriting...


Processing:  /zfsdata/data/david/qligfepv2-BenchmarkExperiments/startFiles/tyk2/protein


STOP qprep ended normally
2024-07-30 08:18:14 | INFO     | QligFEP.CLI.qprep_cli:main:237 - qprep run finished. Check the output `qprep.out` for more information.
2024-07-30 08:18:14 | INFO     | QligFEP.CLI.qprep_cli:main:260 - water.pdb file created.
2024-07-30 08:18:14 | INFO     | QligFEP.CLI.qprep_cli:main:280 - All water molecules are inside the sphere radius.


# TODO:
After running qprep, remove the water molecules from `protein.pdb`.

# Checking for qprep errors:

In [22]:
import subprocess

outqprep_pattern = "*/protein/qprep/qprep.out"
p = subprocess.Popen(
    " ".join(["grep", "-winr", "error", outqprep_pattern]),
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE,
    shell=True,
    text=True,
)
stdout, stderr = p.communicate()
stdout.split("\n")

['cdk2/protein/qprep/qprep.out:85:>>>>> ERROR: Residue number   162 is of unknown type TPO ',
 'cdk2/protein/qprep/qprep.out:88:>>>>> ERROR: The check of the PDB file failed.',
 '']

# Move perturbation files to directory

In [23]:
from pathlib import Path
from joblib import Parallel, delayed
import shutil


def copyfile(src: Path, dest: Path):
    try:
        shutil.copy(src, dest / src.name)
    except FileNotFoundError as e:
        print(f"error for src: {src}:\n{e}")

In [24]:
targets = [p for p in sorted(Path().glob("*/")) if p.is_dir()]

destpath = Path.cwd().parent / "perturbations"
for target in targets:
    ligands_sdf = sorted((target / "ligands/ligands").glob("*.sdf"))
    ligands_pdb = sorted((target / "ligands/").glob("*.pdb"))
    ligands_lib = sorted((target / "ligands/").glob("*.lib"))
    ligands_prm = sorted((target / "ligands/").glob("*.prm"))
    lomap_file = target / "ligands/ligands/lomap.json"
    protein_file = target / "protein/qprep/protein.pdb"
    noHOH_protfile = target / "protein/qprep/protein_noHOH.pdb"
    water_file = target / "protein/qprep/water.pdb"
    perturbation_root = destpath / target.name
    if not perturbation_root.exists():
        perturbation_root.mkdir(parents=True, exist_ok=True)
    allfiles = (
        ligands_sdf
        + ligands_pdb
        + ligands_lib
        + ligands_prm
        + [
            lomap_file,
            (noHOH_protfile if noHOH_protfile.exists() else protein_file),
            water_file,
        ]
    )
    Parallel(n_jobs=6, backend="threading")(
        delayed(copyfile)(src, perturbation_root) for src in allfiles
    )
    if (
        perturbation_root / "protein_noHOH.pdb"
    ).exists():  # rename it to just protein.pdb
        shutil.move(
            perturbation_root / "protein_noHOH.pdb", perturbation_root / "protein.pdb"
        )

error for src: cdk2/protein/qprep/water.pdb:
[Errno 2] No such file or directory: 'cdk2/protein/qprep/water.pdb'
